In [1]:
from taa.search_and_augment import search_and_augment

# return the augmented train dataset in the form of torch.utils.data.Dataset
augmented_train_dataset = search_and_augment(configfile="./taa/confs/bert_imdb_example.yaml")

/Users/tuan/opt/anaconda3/envs/taa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tuan/Projects/text-autoaugment-demo/taa/search.py:11: DeprecationWarning: The module `ray.tune.trial_runner` has been moved to `ray.tune.execution.trial_runner` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.trial_runner` with `ray.tune.execution.trial_runner`.
  import ray.tune.trial_runner
/Users/tuan/Projects/text-autoaugment-demo/taa/search.py:13: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.se

Make TF-IDF model directory: /home/tuan/Projects/text-autoaugment-demo/model/models/tfidf/imdb


OSError: [Errno 45] Operation not supported: '/home/tuan'

# Use pre-searched policy

# Search policy for dataset

# Search policy for Vietnamese dataset